# Movie Recommendation System 

In [1]:
import numpy as np
import pandas as pd
import difflib
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv(r'C:\Users\user\Downloads\movies.csv')
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
data.shape

(4803, 24)

In [4]:
columns = data.columns

In [5]:
for column in columns:
    print(column)

index
budget
genres
homepage
id
keywords
original_language
original_title
overview
popularity
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
vote_average
vote_count
cast
crew
director


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [7]:
# Selecting relevant features for recommendation 
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features 

['genres', 'keywords', 'tagline', 'cast', 'director']

In [8]:
# filling missing data with empty string
for feature in selected_features:
    data[feature] = data[feature].fillna('')

In [9]:
# Concatenating data for vector conversion
combined_features = data['genres']+' '+data['keywords']+' '+data['tagline']+' '
+data['cast']+' '+data['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801                            A New Yorker in Shanghai 
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
# Converting textual data into feature vectors 
vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 4498)	0.35977146596126014
  (0, 4348)	0.11969800165259886
  (0, 7005)	0.16951497952634928
  (0, 6257)	0.09428706610088097
  (0, 2102)	0.3176552747616365
  (0, 5741)	0.2828344413384406
  (0, 1321)	0.33000816200564936
  (0, 6802)	0.16592099630102675
  (0, 5793)	0.44903494633034546
  (0, 2576)	0.21772343390141816
  (0, 1244)	0.2968078745280208
  (0, 1585)	0.2828344413384406
  (0, 2370)	0.13702838223474842
  (0, 5427)	0.13702838223474842
  (0, 2296)	0.14687069071321432
  (0, 190)	0.11927285589220428
  (0, 155)	0.10392045738902624
  (1, 646)	0.2256283604141521
  (1, 2078)	0.2363184185838419
  (1, 455)	0.1983798683793789
  (1, 3637)	0.15190171604782024
  (1, 4374)	0.14471052044386193
  (1, 3724)	0.14052089954370584
  (1, 1361)	0.26118368008546655
  (1, 6401)	0.3180638946881614
  :	:
  (4799, 4348)	0.11911590179532587
  (4799, 6257)	0.0938285414224214
  (4800, 5621)	0.3545557766157617
  (4800, 6187)	0.3488844346046621
  (4800, 4193)	0.37207260176999274
  (4800, 2416)	0.27494012230371234

In [12]:
# Computing similarity score for the features
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.13264148 0.02918063 ... 0.         0.         0.        ]
 [0.13264148 1.         0.06857325 ... 0.08092515 0.         0.        ]
 [0.02918063 0.06857325 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.08092515 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [13]:
movie_name = input('Enter your movie: ')

Enter your movie: fast and furious


In [14]:
list_of_all_movies = list(data['title'])
list_of_all_movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [15]:
# finding close match of movie entered by the user 
close_matches = difflib.get_close_matches(movie_name, list_of_all_movies)
close_matches

['The Fast and the Furious', '2 Fast 2 Furious']

In [16]:
close_match = close_matches[0]
print(close_match)

The Fast and the Furious


In [17]:
# getting the index of the close match
index_of_movie = data[data['title'] == close_match]['index'].values[0]
index_of_movie

99

In [18]:
# getting list of similar movies 
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.010620403817220557), (1, 0.07678279397557469), (2, 0.030088870339807193), (3, 0.060233357566515565), (4, 0.010998941165259414), (5, 0.04395723891173866), (6, 0.0), (7, 0.011586462440965041), (8, 0.0), (9, 0.011638652243239907), (10, 0.014117437548624028), (11, 0.08560971738600232), (12, 0.01184209903147139), (13, 0.010675408274482158), (14, 0.010610002169680164), (15, 0.0), (16, 0.010445586337896254), (17, 0.07281467036017454), (18, 0.10647533705739672), (19, 0.008603103831139935), (20, 0.01103928908829941), (21, 0.007864997985493527), (22, 0.0), (23, 0.0), (24, 0.009503605991029831), (25, 0.010155033966621277), (26, 0.012488114044472375), (27, 0.06459301093399641), (28, 0.022444369819906745), (29, 0.023309451221359294), (30, 0.04026090748205521), (31, 0.010588752181953253), (32, 0.0), (33, 0.021735028626380527), (34, 0.0), (35, 0.012488388826004428), (36, 0.010777267420531408), (37, 0.0), (38, 0.011509729277805445), (39, 0.011104192750743971), (40, 0.13266973429956697), (41, 0.

In [19]:
sorted_similar_movies = sorted(similarity_score, key =lambda x:x[1], reverse=True  )
print(sorted_similar_movies)

[(99, 1.0), (2548, 0.5133844503265883), (500, 0.3640960221950352), (405, 0.2927823665136227), (2682, 0.2616270703744757), (658, 0.2483433434373913), (1186, 0.23479711924816477), (566, 0.22704925160597209), (2154, 0.21959845747293796), (1399, 0.21750525469834345), (44, 0.20532504668772253), (2074, 0.20193621433926875), (3444, 0.19674515280858007), (273, 0.1926513995170443), (1245, 0.19093005883956218), (3420, 0.19071987255628586), (2604, 0.19053421817142416), (3522, 0.18901619448497514), (3714, 0.18740306058512976), (4256, 0.18740306058512976), (4433, 0.18713254035648472), (763, 0.18597623961096502), (747, 0.18257361866271085), (2931, 0.18044448758233822), (706, 0.17731720395511313), (629, 0.17504722491991867), (3274, 0.174710323082479), (3359, 0.1745473543263832), (935, 0.17282903735271857), (2741, 0.17199380862062358), (225, 0.17167417170659485), (2371, 0.17109096729453296), (560, 0.16899952571992352), (4467, 0.16511028282374907), (376, 0.1646814587847932), (2492, 0.16429330791798494)

In [20]:
# printing the names of similar movies based on index 

print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = data[data['index'] == index]['title'].values[0]
    if i < 31:
        print(i, '.', title_from_index)
        i+=1

Movies suggested for you: 

1 . The Fast and the Furious
2 . Malibu's Most Wanted
3 . 2 Fast 2 Furious
4 . The Fast and the Furious: Tokyo Drift
5 . Poetic Justice
6 . Death Race
7 . The Final Destination
8 . Cars
9 . Street Kings
10 . In Time
11 . Furious 7
12 . The Transporter
13 . Grease
14 . Gone in Sixty Seconds
15 . Colombiana
16 . Boyz n the Hood
17 . Here On Earth
18 . Crossover
19 . Exiled
20 . Checkmate
21 . L!fe Happens
22 . The Soloist
23 . Gangster Squad
24 . Corky Romano
25 . Days of Thunder
26 . Need for Speed
27 . Showdown in Little Tokyo
28 . In Too Deep
29 . Herbie Fully Loaded
30 . National Lampoon's Loaded Weapon 1


## Recommendion System

In [23]:
movie_name = input('Enter your movie: ')

list_of_all_movies = list(data['title'])

close_matches = difflib.get_close_matches(movie_name, list_of_all_movies)

close_match = close_matches[0]

index_of_movie = data[data['title'] == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]))

sorted_similar_movies = sorted(similarity_score, key =lambda x:x[1], reverse=True  )

print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = data[data['index'] == index]['title'].values[0]
    if i < 31:
        print(i, '.', title_from_index)
        i+=1

Enter your movie: blood and treasure
Movies suggested for you: 

1 . Blood and Wine
2 . The Punisher
3 . Code of Honor
4 . Firestorm
5 . 11:14
6 . Doubt
7 . Sin City: A Dame to Kill For
8 . Wonderland
9 . Zulu
10 . N-Secure
11 . Dutch Kills
12 . Les Misérables
13 . Closer to the Moon
14 . Gangster's Paradise: Jerusalema
15 . The Blue Room
16 . Z Storm
17 . Perrier’s Bounty
18 . In Too Deep
19 . 3000 Miles to Graceland
20 . Containment
21 . Mission: Impossible - Ghost Protocol
22 . Black Water Transit
23 . Deadline - U.S.A.
24 . Ajami
25 . Water & Power
26 . El Rey de Najayo
27 . Bangkok Dangerous
28 . In Cold Blood
29 . The Swindle
30 . Gangster Squad
